In [1]:
import pandas as pd
import numpy as np

In [2]:
game_df = pd.read_csv("raw_data/Twitch_game_data.csv")
global_df = pd.read_csv("raw_data/Twitch_global_data.csv")
streamers_df = pd.read_csv("raw_data/twitchdata-update.csv")

In [3]:
def dateToDateTime(dataframe):
    df = dataframe.copy()
    df["day"] = 1
    df = df.set_index(pd.to_datetime(df[["year", "Month", "day"]]))
    df = df.drop(["year", "Month", "day"], axis=1)
    return df

## Global viewers chart

In [4]:
Viewers = global_df[["year","Month","Avg_viewers"]]

In [5]:
Viewers = dateToDateTime(Viewers)

In [6]:
Viewers.to_csv("data/global_viewers.csv")

## Games views for ridgeplot

In [7]:
non_games= ["Just Chatting","Talk Shows & Podcasts","Poker","Music","Creative","Pools, Hot Tubs, and Beaches","Science & Technology","Special Events","Art","Politics","Travel & Outdoors","Sports","ASMR","Food & Drink","Makers & Crafting","Beauty & Body Art","Software and Game Development"]

for ng in non_games:

    game_df.drop(game_df[game_df["Game"] == ng].index, inplace=True)


In [8]:
game_avg = game_df.copy()
game_avg.drop(columns=["Rank","Hours_watched","Hours_Streamed","Peak_viewers","Peak_channels","Streamers","Avg_channels","Avg_viewer_ratio"] ,inplace=True)

In [9]:
game_avg["day"] = 1
game_avg["date"] = pd.to_datetime(game_avg[["Year", "Month", "day"]])
game_avg = game_avg.drop(["Year", "Month", "day"], axis=1)

In [10]:
top_games = game_avg.groupby(['Game'])['Avg_viewers'].sum().sort_values(ascending=False)[:10].index.to_list()

In [11]:
games = []

for game in top_games:
    games.append(game_avg[game_avg["Game"] == game])

In [12]:
for g in games:
    g.rename(columns={"Avg_viewers" : g.iloc[0]["Game"]},inplace=True)
    g.drop(columns="Game", inplace=True)

d:\Ecole\master\twitch-to-the-moon\.env\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
d:\Ecole\master\twitch-to-the-moon\.env\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
from functools import reduce

pd.concat(games)

top_games_avg_viewers = reduce(lambda  left,right: pd.merge(left,right, on="date", how="outer"), games).fillna(0)

In [14]:
top_games_avg_viewers.to_csv("data/top_games_viewers.csv")

for i,game in enumerate(games):
    game.to_csv("data/"+ games[i]["Game"].iloc[0] +".csv")

## Average viewers of all games

In [15]:
all_games = game_avg.groupby(['Game'])['Avg_viewers'].sum().sort_values(ascending=False).index.to_list()

games = []

for game in all_games:
    games.append(game_avg[game_avg["Game"] == game])

In [16]:
for g in games:
    g.rename(columns={"Avg_viewers" : g.iloc[0]["Game"]},inplace=True)
    g.drop(columns="Game", inplace=True)


In [17]:
all_games_avg_view = reduce(lambda  left,right: pd.merge(left,right, on="date", how="outer"), games).fillna(0)

In [18]:
all_games_avg_view.to_csv("data/all_games_viewers.csv")

## Domination of the Game

In [25]:
global_avg_views = all_games_avg_view.drop(columns="date").sum(axis=1)

In [27]:
all_games_avg_view["all_games"] = global_avg_views

In [45]:
all_games_domination = pd.DataFrame()
for c in all_games_avg_view.columns:
    if c in ["date","all_games"]:
        all_games_domination[c] = all_games_avg_view[c]
    else:
        all_games_domination[c] = (all_games_avg_view[c] / all_games_avg_view["all_games"])*100

C:\Users\ma2s\AppData\Local\Temp/ipykernel_5496/405434939.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  all_games_domination[c] = (all_games_avg_view[c] / all_games_avg_view["all_games"])*100
C:\Users\ma2s\AppData\Local\Temp/ipykernel_5496/405434939.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  all_games_domination[c] = all_games_avg_view[c]


In [47]:
all_games_domination.drop(columns="all_games", inplace=True)

In [48]:
all_games_domination.to_csv("data/all_games_domination.csv")

## Radar chart metrics